# YugabyteDB: RF3 in Secure Mode

__NOTE: This notebook requires the Bash Kernel__

## Introduction

There are multiple options available to run YugabyteDB locally.  For simple, single node use cases, Docker is an exceptionally good tool.  However, for multi-node enviornments, Docker can be a bit trickier to configure correctly due to network setup.

The next option is to use the `yugabyted` command line tool outside Docker.  This tool is bundled with the YugabyteDB distribution and can be easily downloaded and unpacked to a local machine.  Typically, I recommend downloading the latest STS from the Yugabyte [downloads](https://download.yugabyte.com/#/) page.  Pick your preferred distribution (Mac or Linux) and download it with `curl`/`wget` as recommended and untar to a desired folder.

In [9]:
# Initialize secrets from the local .env file here
source .env
echo $MY_SECRET

This is my secret, please don't upload it to GitHub!


In [1]:
mkdir ~/yugabyte; cd $_
#wget https://downloads.yugabyte.com/releases/2024.1.3.0/yugabyte-2024.1.3.0-b105-linux-x86_64.tar.gz
#tar xfz yugabyte-2024.1.3.0-b105-linux-x86_64.tar.gz
cd yugabyte-2024.1.3.0/
./bin/post_install.sh

mkdir: cannot create directory ‘/home/lurtz/yugabyte’: File exists
OpenSSL binary: ./bin/../bin/../bin/openssl
FIPS module: ./bin/../bin/../lib/ossl-modules/fips.so
HMAC : (Module_Integrity) : Pass
SHA1 : (KAT_Digest) : Pass
SHA2 : (KAT_Digest) : Pass
SHA3 : (KAT_Digest) : Pass
TDES : (KAT_Cipher) : Pass
AES_GCM : (KAT_Cipher) : Pass
AES_ECB_Decrypt : (KAT_Cipher) : Pass
RSA : (KAT_Signature) : RNG : (Continuous_RNG_Test) : Pass
Pass
ECDSA : (PCT_Signature) : Pass
ECDSA : (PCT_Signature) : Pass
DSA : (PCT_Signature) : Pass
TLS13_KDF_EXTRACT : (KAT_KDF) : Pass
TLS13_KDF_EXPAND : (KAT_KDF) : Pass
TLS12_PRF : (KAT_KDF) : Pass
PBKDF2 : (KAT_KDF) : Pass
SSHKDF : (KAT_KDF) : Pass
KBKDF : (KAT_KDF) : Pass
HKDF : (KAT_KDF) : Pass
SSKDF : (KAT_KDF) : Pass
X963KDF : (KAT_KDF) : Pass
X942KDF : (KAT_KDF) : Pass
HASH : (DRBG) : Pass
CTR : (DRBG) : Pass
HMAC : (DRBG) : Pass
DH : (KAT_KA) : Pass
ECDH : (KAT_KA) : Pass
RSA_Encrypt : (KAT_AsymmetricCipher) : Pass
RSA_Decrypt : (KAT_AsymmetricCipher) : 

## Validate the Installation

Assuming the previous commands ran successfully, you should have downloaded and unpacked the YugabyteDB distribution.  The final `post_install.sh` script is used to validate the deployent. The last output line should read, `INSTALL PASSED`.

Confirm the version running the `yugabyted` command with the `version` option:

In [2]:
./bin/yugabyted version


----------------------------------------------------------------------
|                              Version                               |
----------------------------------------------------------------------
| Version        : 2024.1.3.0-b105                                   |
| Build Time     : 03 Oct 2024 18:48:31 UTC                          |
| Build Hash     : 0b04fe41caf79c06cfce56cd80731e072853fe3b          |
----------------------------------------------------------------------



## Create Certificates (Optional)

To run locally with TLS (SSL) enabled, first create the certs for a 3 node REF cluster using [generate_server_certs](). This command will generate certs for each of the nodes using the IP address they will be running on.

On Linux we will use multiple local 127.0.0.x hosts:

In [12]:
./bin/yugabyted cert generate_server_certs --hostnames=127.0.0.1,127.0.0.2,127.0.0.3

⚠ WARNING: Hostnames 127.0.0.1,127.0.0.2,127.0.0.3 already have their certifcates generated according to root-ca certs database index. Please use the previously generated certs. Skipping these hostnames....

+-----------------------------------------------------------------------------------------------------+
|                                              yugabyted                                              |
+-----------------------------------------------------------------------------------------------------+
| Status                        : No certs generated. All hostnames already have certs generated.     |
| Server node certs path        : Certs can be found at: /home/lurtz/var/generated_certs/<hostname>   |
+-----------------------------------------------------------------------------------------------------+



## Launch the first node

Before we start the first node of the cluster, lets set up a few enviornment variables that will help with configuring the YugabyteDB processes.
Review confiugration options here:

* [YB Master config](https://docs.yugabyte.com/preview/reference/configuration/yb-master/)
* [TServer config](https://docs.yugabyte.com/preview/reference/configuration/yb-tserver/)

In [19]:
export MASTER_FLAGS=""
export TSERVER_FLAGS="yb_enable_read_committed_isolation=true"

For each of the three nodes we want to start, we use the `yugabyted` command with specific options to idenify each node (including the certs we generated earlier):

In [10]:
export CERTS_DIR="~/var/generated_certs/127.0.0.1/"
yugabyted start --advertise_address=127.0.0.1 --base_dir=~/var/ybd-gcp1 --cloud_location=gcp.us-east1.us-east1-c --secure --certs_dir="${CERTS_DIR}" --tserver_flags "${TSERVER_FLAGS}"

Starting yugabyted...
✅ YugabyteDB Started                  
✅ Encryption in Transit and Password Authentication enabled    
✅ UI ready         
✅ Data placement constraint successfully verified                 

⚠ WARNINGS:
- Transparent hugepages disabled. Please enable transparent_hugepages.

Please review the following docs and rerun the start command:
- Quick start for Linux: https://docs.yugabyte.com/preview/quick-start/linux/

+-------------------------------------------------------------------------------------------------------------+
|                                                  yugabyted                                                  |
+-------------------------------------------------------------------------------------------------------------+
| Status              : Running.                                                                              |
| Replication Factor  : 1                                                                                     |
| 

## Retrieve the generated password

Take special note of the last line of the `yugabyted` output as it indicates where to find the generated password. Echo it out to the console:

In [18]:
cat /home/lurtz/var/ybd-gcp1/data/yugabyted_credentials.txt

YSQL Credentials:
Username: yugabyte
Password: URYuo4eeQ2qM
Database: yugabyte

YCQL Credentials:
Username: cassandra
Password: URYuo4eeQ2qM
Keyspace: None


In [11]:
export CERTS_DIR="~/var/generated_certs/127.0.0.2/"
yugabyted start --advertise_address=127.0.0.2 --base_dir=~/var/ybd-gcp2 --cloud_location=gcp.us-east1.us-east1-d --secure --certs_dir="${CERTS_DIR}" --tserver_flags "${TSERVER_FLAGS}" --join=127.0.0.1

Fetching configs from join IP...
Starting yugabyted...
✅ YugabyteDB Started                  
✅ Node joined a running cluster with UUID f250ee44-b9fe-4203-b2b3-25818f385b18
✅ Encryption in Transit and Password Authentication enabled    
✅ UI ready         
✅ Data placement constraint successfully verified                 

⚠ WARNINGS:
- Transparent hugepages disabled. Please enable transparent_hugepages.

Please review the following docs and rerun the start command:
- Quick start for Linux: https://docs.yugabyte.com/preview/quick-start/linux/

+-------------------------------------------------------------------------------------------------------------+
|                                                  yugabyted                                                  |
+-------------------------------------------------------------------------------------------------------------+
| Status              : Running.                                                                              |
| 

In [12]:
export CERTS_DIR="~/var/generated_certs/127.0.0.3/"
yugabyted start --advertise_address=127.0.0.3 --base_dir=~/var/ybd-gcp3 --cloud_location=gcp.us-east1.us-east1-f --secure --certs_dir="${CERTS_DIR}" --tserver_flags "${TSERVER_FLAGS}" --join=127.0.0.1

Fetching configs from join IP...
Starting yugabyted...
✅ YugabyteDB Started                  
✅ Node joined a running cluster with UUID f250ee44-b9fe-4203-b2b3-25818f385b18
✅ Encryption in Transit and Password Authentication enabled    
✅ UI ready         
✅ Data placement constraint successfully verified                 

⚠ WARNINGS:
- Transparent hugepages disabled. Please enable transparent_hugepages.

Please review the following docs and rerun the start command:
- Quick start for Linux: https://docs.yugabyte.com/preview/quick-start/linux/

+-------------------------------------------------------------------------------------------------------------+
|                                                  yugabyted                                                  |
+-------------------------------------------------------------------------------------------------------------+
| Status              : Running.                                                                              |
| 

In [13]:
yugabyted status --base_dir=~/var/ybd-gcp1

tching status...
+-------------------------------------------------------------------------------------------------------------+
|                                                  yugabyted                                                  |
+-------------------------------------------------------------------------------------------------------------+
| Status              : Running.                                                                              |
| Replication Factor  : 3                                                                                     |
| Security Features   : Encryption-in-transit, Password Authentication                                        |
| YugabyteDB UI       : http://127.0.0.1:15433                                                                |
| JDBC                : jdbc:postgresql://127.0.0.1:5433/yugabyte?user=yugabyte&password=<DB_PASSWORD>        |
| YSQL                : bin/ysqlsh   -U yugabyte -d yugabyte                           

Command 'from' not found, but can be installed with:
sudo apt install mailutils


## Generate a new Java Trust Store (JKS)

For Java, use the `keytool` command (it should be bundled with the Java installation). The following command will create a new trust store (a special file) and import the root certificate authority (CA) from the orignal generated certs.

In [23]:
keytool -importcert -trustcacerts -file ~/var/generated_certs/root-ca/ca.crt -alias yuabyteCA -keystore yugabyte.jks -storepass change1t -noprompt

Certificate was added to keystore


Tecnically, you have a common truststore that is generically used by Java that contains standard CAs but since the CA we generated earlier is local and will probably be thrown away at some point, it is easier (and cleaner) to just keep a separate truststore for this specific use case.

In [24]:
keytool -list -keystore yugabyte.jks -storepass change1t

Keystore type: PKCS12
Keystore provider: SUN

Your keystore contains 1 entry

yuabyteca, Oct 24, 2024, trustedCertEntry, 
Certificate fingerprint (SHA-256): 7A:71:63:2E:FC:6B:6F:3E:70:AD:28:FF:E8:E2:CB:AB:EE:36:BB:0F:01:68:B7:15:5A:F5:D8:62:BA:0A:78:98


This is a secret, don't upload it to GitHub
